In [2]:
import numpy as np
import plotly.graph_objects as go

#x,y,z = np.genfromtxt(r'dat.txt', unpack=True)
x = np.arange(100)
y = np.random.random(100)
z = np.random.random(100)

# Create figure
fig = go.Figure(
    data=[go.Scatter3d(x=[], y=[], z=[],
                     mode="markers",marker=dict(color="red", size=10))])
    
fig.update_layout(    
    scene = dict(
        
        xaxis=dict(range=[min(x), max(x)], autorange=False),
        yaxis=dict(range=[min(y), max(y)], autorange=False),
        zaxis=dict(range=[min(z), max(z)], autorange=False),
        )),


frames = [go.Frame(data= [go.Scatter3d(
                                       x=x[[k]], 
                                       y=y[[k]],
                                       z=z[[k]])],
                   
                   traces= [0],
                   name=f'frame{k}'      
                  )for k  in  range(len(x))]
fig.update(frames=frames),




fig.update_layout(updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, dict(frame=dict(redraw=True,fromcurrent=True, mode='immediate'))      ])])])


fig.show()


In [12]:
import pandas as pd
import io
import os
import glob
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np


@dataclass
class MocapLowerLandmark :
    LANK: np.ndarray = None # 0
    LASI: np.ndarray = None # 1
    LHEE: np.ndarray = None # 2
    LKNE: np.ndarray = None # 3
    LPSI: np.ndarray = None # 4
    LTHI: np.ndarray = None # 5
    LTIB: np.ndarray = None # 6
    LTOE: np.ndarray = None # 7
    RANK: np.ndarray = None # 8
    RASI: np.ndarray = None # 9
    RHEE: np.ndarray = None # 10
    RKNE: np.ndarray = None # 11
    RPSI: np.ndarray = None # 12
    RTHI: np.ndarray = None # 13
    RTIB: np.ndarray = None # 14
    RTOE: np.ndarray = None # 15
    
    CONN_PATH_LIST = [
         7,  2,  0,  3,  1,  4, 
        12,  9, 11,  8, 10, 15
    ]

    @staticmethod
    def calcKneeDegrees(mocap_df, idx = 0) :
        raw_val = mocap_df.loc[:, ~mocap_df.columns.isin(["Frame", "Time (Seconds)"]) ].to_numpy()
        raw_val = raw_val[idx].reshape(16, 3)

        rthai = raw_val[9] - raw_val[11]
        rcalf = raw_val[8] - raw_val[11]

        lthai = raw_val[1] - raw_val[3]
        lcalf = raw_val[0] - raw_val[3]

        rthai_unit = rthai / np.linalg.norm(rthai)
        rcalf_unit = rcalf / np.linalg.norm(rcalf)

        lthai_unit = lthai / np.linalg.norm(lthai)
        lcalf_unit = lcalf / np.linalg.norm(lcalf)

        rangle = np.arccos(np.dot(rthai_unit, rcalf_unit))
        langle = np.arccos(np.dot(lthai_unit, lcalf_unit))

        return langle, rangle

    @staticmethod
    def df2landmark(mocap_df, idx = 0) :
        raw_val = mocap_df.loc[:, ~mocap_df.columns.isin(["Frame", "Time (Seconds)"]) ].to_numpy()
        return raw_val[idx].reshape(16, 3)


def read_whole_data(trial_idx, experiment_idx) :
    FORCE_PLATE_1_PATH = f"/Volumes/HJP/CODES/mocap/data/d1/230626_Group7_trial{trial_idx}_{experiment_idx:03}_forceplate_1.csv"
    FORCE_PLATE_2_PATH = f"/Volumes/HJP/CODES/mocap/data/d1/230626_Group7_trial{trial_idx}_{experiment_idx:03}_forceplate_2.csv"
    MOCAP_PATH = f"/Volumes/HJP/CODES/mocap/data/d1/230626_Group7_trial{trial_idx}_{experiment_idx:03}.csv"
    EMG_PATH = f"/Volumes/HJP/CODES/mocap/data/d1/230626_Group7_trial{trial_idx}_{experiment_idx:03}_Trigno_2883.csv"

    
    with open(FORCE_PLATE_1_PATH, "r") as fp :
        lines = fp.readlines()
    raw_text = "".join(lines)
    table = raw_text.split("\n\n")[-1]
    fp1_df = pd.read_csv(io.StringIO(table))

    with open(FORCE_PLATE_2_PATH, "r") as fp :
        lines = fp.readlines()
    raw_text = "".join(lines)
    table = raw_text.split("\n\n")[-1]
    fp2_df = pd.read_csv(io.StringIO(table))

    with open(EMG_PATH, "r") as fp :
        lines = fp.readlines()
    raw_text = "".join(lines)
    table = raw_text.split("\n\n")[-1]
    emg_df = pd.read_csv(io.StringIO(table))

    with open(MOCAP_PATH, "r") as fp :
        lines = fp.readlines()
    mocap_df = pd.read_csv(io.StringIO( ''.join(lines[6:]) ))
    
    return mocap_df, emg_df, fp1_df, fp2_df

mocap_df, emg_df, fp1_df, fp2_df = read_whole_data(2, 1)

In [39]:
import numpy as np
import plotly.graph_objects as go

#x,y,z = np.genfromtxt(r'dat.txt', unpack=True)
x = np.arange(100)
y = np.random.random(100)
z = np.random.random(100)

# Create figure
fig = go.Figure(
    data = go.Scatter3d(
        x=[], y=[], z=[],
        mode="markers",
        marker=dict(color="red", size=10)
    )
)

# Frames
frames = [
    go.Frame(
        data = [
            go.Scatter3d(
                x=x[[k]],
                y=y[[k]],
                z=z[[k]]
            )
        ],
        traces= [0],
        name=f'frame{k}'      
    )for k  in  range(len(x)-1)
]
fig.update(frames=frames)


def frame_args(duration):
    return {
        "frame": {"duration": duration},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "linear"},
    }


sliders = [
    {
        "pad": {"b": 10, "t": 60},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [
            {
                "args": [[f.name], frame_args(0)],
                "label": str(k),
                "method": "animate",
            } for k, f in enumerate(fig.frames)
        ]
    }
]

fig.update_layout(
    updatemenus = [
        {
            "buttons":[
                {
                    "args": [None, frame_args(50)],
                    "label": "Play", 
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],
                    "label": "Pause", 
                    "method": "animate",
                }
            ],    
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }
    ],
    sliders=sliders
)

fig.update_layout(
    scene = dict(
        xaxis=dict(range=[min(x), max(x)],autorange=False),
        yaxis=dict(range=[min(y), max(y)], autorange=False),
        zaxis=dict(range=[min(z), max(z)], autorange=False)
    )
)

fig.update_layout(sliders=sliders)
fig.show()

In [70]:
MocapLowerLandmark.df2landmark(mocap_df, 100)

def plot3d_with_slider(mocap_df) :
    n_frames = len(mocap_df)
    points_list = np.array(list(map(
        lambda idx : MocapLowerLandmark.df2landmark(
            mocap_df, idx
        ),
        range(n_frames)
    )))
    
    points_list_ = np.nan_to_num(points_list.copy(), 0.1)

    xmin = points_list_[:, :, 0].min()
    xmax = points_list_[:, :, 0].max()
    ymin = points_list_[:, :, 1].min()
    ymax = points_list_[:, :, 1].max()
    zmin = points_list_[:, :, 2].min()
    zmax = points_list_[:, :, 2].max()

    
    connection_idx_list = MocapLowerLandmark.CONN_PATH_LIST
    line_color_list = ['red'] * (len(connection_idx_list))
    
    
    fig = go.Figure(
        data = go.Scatter3d(
            x=[], y=[], z=[],
            mode = 'lines+markers+text',
            marker=dict(color="red", size=2)
        )
    )
    frame_list = [
        go.Frame(
            data = list(map(
                lambda i, j, c : go.Scatter3d(
                    x = points[[i, j], 0],
                    y = points[[i, j], 1],
                    z = points[[i, j], 2],
                    mode = 'lines+markers+text',
                    line = dict(
                        color = c
                    ),
                    marker = dict(
                        color = c,
                        size = 2
                    )
                ),
                connection_idx_list[:-1],
                connection_idx_list[1:],
                line_color_list
            )),
            traces=[0],
            name=f"frame{fidx}"
        )
        for fidx, points in enumerate(points_list)
    ]
    fig.update(frames = frames)

    def frame_args(duration):
        return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

    sliders = [
        {
            "pad": {"b": 10, "t": 60},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": [
                {
                    "args": [[f.name], frame_args(0)],
                    "label": str(k),
                    "method": "animate",
                } for k, f in enumerate(fig.frames)
            ]
        }
    ]

    fig.update_layout(
        updatemenus = [
            {
                "buttons":[
                    {
                        "args": [None, frame_args(50)],
                        "label": "Play", 
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "Pause", 
                        "method": "animate",
                    }
                ],    
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
        ],
        sliders=sliders
    )

    layout = go.Layout(
        scene = dict(
            camera = dict(
                eye = dict(x=-1, y = -1, z = 1)
            ),
            aspectmode="data"
        )
    )
    fig.layout = layout

    max_range = max(xmax-xmin, ymax-ymin, zmax, zmin)
    #max_range = np.max(np.ptp(points_list[0], axis=0))
    #xmin, ymin, zmin = points_list[0].min(axis=0)

    fig.update_layout(scene=dict(aspectmode='cube'))
    fig.update_layout(
        scene=dict(
            xaxis = dict(range=[xmin-0.1, xmin + max_range+ 0.1]),
            yaxis = dict(range=[ymin-0.1, ymin + max_range+ 0.1]),
            zaxis = dict(range=[zmin-0.1, zmin + max_range+ 0.1])
        )
    )



    fig.update_layout(sliders=sliders)
    fig.show()

plot3d_with_slider(mocap_df)

In [56]:
n_frames = len(mocap_df)
points_list = np.array(list(map(
    lambda idx : MocapLowerLandmark.df2landmark(
        mocap_df, idx
    ),
    range(n_frames)
)))

points_list_ = np.nan_to_num(points_list.copy(), 0.1)

xmin = points_list_[:, :, 0].min()
xmax = points_list_[:, :, 0].max()
ymin = points_list_[:, :, 1].min()
ymax = points_list_[:, :, 1].max()
zmin = points_list_[:, :, 2].min()
zmax = points_list_[:, :, 2].max()

xmin, xmax, ymin, ymax, zmin, zmax

(0.0, 0.575867, 0.0, 0.993897, 0.0, 0.836932)

In [58]:
max(1, 2, 3)

3

In [69]:
##

import numpy as np
import plotly.graph_objects as go

n_frames = len(mocap_df)
points_list = np.array(list(map(
    lambda idx : MocapLowerLandmark.df2landmark(
        mocap_df, idx
    ),
    range(n_frames)
)))
points_list_ = np.nan_to_num(points_list.copy(), 0.1)

xmin = points_list_[:, :, 0].min()
xmax = points_list_[:, :, 0].max()
ymin = points_list_[:, :, 1].min()
ymax = points_list_[:, :, 1].max()
zmin = points_list_[:, :, 2].min()
zmax = points_list_[:, :, 2].max()

connection_idx_list = MocapLowerLandmark.CONN_PATH_LIST
line_color_list = ['red'] * (len(connection_idx_list))

lidx = 0
ridx = 100
jidx = 5
x = points_list[lidx:ridx, jidx, 0]
y = points_list[lidx:ridx, jidx, 1]
z = points_list[lidx:ridx, jidx, 2]

print(x)
print(y)
print(z)

# Create figure
fig = go.Figure(
    data = go.Scatter3d(
        x=[], y=[], z=[],
        mode="markers",
        marker=dict(color="red", size=10)
    )
)


# Frames
'''
frames = [
    go.Frame(
        data = [
            go.Scatter3d(
                x=x[[k]],
                y=y[[k]],
                z=z[[k]]
            )
        ],
        traces= [0],
        name=f'frame{k}'      
    )for k  in  range(len(x)-1)
]
fig.update(frames=frames)
'''
frames = [
    go.Frame(
        data = list(map(
            lambda i, j, c : go.Scatter3d(
                x = points[[i, j], 0],
            )
        ))
        
        
        
        [
            go.Scatter3d(
                x=x[[k]],
                y=y[[k]],
                z=z[[k]]
            )
        ],
        traces= [0],
        name=f'frame{k}'      
    )
    for k, points in enumerate(points_list)

]
fig.update(frames=frames)



def frame_args(duration):
    return {
        "frame": {"duration": duration},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "linear"},
    }


sliders = [
    {
        "pad": {"b": 10, "t": 60},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [
            {
                "args": [[f.name], frame_args(0)],
                "label": str(k),
                "method": "animate",
            } for k, f in enumerate(fig.frames)
        ]
    }
]

fig.update_layout(
    updatemenus = [
        {
            "buttons":[
                {
                    "args": [None, frame_args(50)],
                    "label": "Play", 
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],
                    "label": "Pause", 
                    "method": "animate",
                }
            ],    
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }
    ],
    sliders=sliders
)

fig.update_layout(
    scene = dict(
        xaxis=dict(range=[min(x), max(x)],autorange=False),
        yaxis=dict(range=[min(y), max(y)], autorange=False),
        zaxis=dict(range=[min(z), max(z)], autorange=False)
    )
)


fig.update_layout(sliders=sliders)
fig.show()

[0.291835 0.291782 0.291747 0.291769 0.291789 0.291778 0.291791 0.291785
 0.291772 0.291743 0.291762 0.291784 0.291831 0.291831 0.291835 0.291847
 0.291867 0.291927 0.292021 0.292167 0.292271 0.292371 0.292398 0.29243
 0.292443 0.292458 0.292488 0.292542 0.292634 0.292726 0.292817 0.292879
 0.292928 0.29294  0.292953 0.292943 0.292965 0.293023 0.293099 0.293256
 0.293415 0.29356  0.293643 0.293695 0.293676 0.293658 0.293628 0.293595
 0.293581 0.293632 0.293722 0.293779 0.29386  0.293908 0.293962 0.293985
 0.294003 0.294055 0.294106 0.294127 0.294084 0.294039 0.293953 0.293903
 0.293872 0.29388  0.293958 0.294096 0.294266 0.294433 0.294554 0.294652
 0.2947   0.294699 0.294649 0.294577 0.294498 0.294414 0.294387 0.294354
 0.294373 0.294362 0.294288 0.29406  0.293728 0.293312 0.292761 0.292101
 0.291326 0.290763 0.290222 0.289733 0.289369 0.289143 0.288933 0.28849
 0.287963 0.287156 0.286088 0.284724]
[0.59291  0.592881 0.592851 0.592873 0.592861 0.592806 0.592822 0.59286
 0.592851 0.5928

In [64]:
points_list.shape

(339, 16, 3)

In [68]:
points_list[0:100][0][0].shape

(3,)